In [95]:
autumn['DATE'] = pd.to_datetime(autumn['DATE'])

In [236]:
# Query to combine the overall metrics with the individual table
def quer(y):
    return'''select a.DATE, 
    a.TMAX, 
    a.TMIN,  
    a.YEAR, 
    a.MONTH, 
    a.DAY, 
    b.over_max, 
    b.over_min 
    from autumn a join maxminall b
    on a.MONTH || a.DAY = b.concat WHERE YEAR = ''' + str(y)

#pd.read_sql(quer, conn)

In [2]:
autumn = pd.read_csv('autumn2.csv')

In [4]:
autumn['RANGE'] = autumn['TMAX'] - autumn['TMIN']

In [5]:
#maybe this is the database?
conn = sqlite3.connect('weather_data.db')
autumn.to_sql('lou_weather',conn)

/Users/ryanpierson/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:2127: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)


ValueError: Table 'lou_weather' already exists.

In [8]:
autumn.loc[autumn['RANGE'].idxmax()]
autumn.loc[autumn['RANGE'].idxmin()]

DATE     2010-10-09
TMAX             87
TMIN             47
RANGE            40
YEAR           2010
MONTH            10
WEEK             40
DAY               9
Name: 38, dtype: object

In [199]:
from bokeh.layouts import gridplot
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
import pandas as pd
import sqlite3

output_notebook()
#Gets the distinct years from the database
query = "Select Distinct YEAR  from autumn"

yr = pd.read_sql(query, conn)

# Query to combine the overall metrics with the individual table
def quer(y):
    return'''select a.DATE, 
    a.TMAX, 
    a.TMIN,  
    a.YEAR, 
    a.MONTH, 
    a.DAY, 
    b.over_max, 
    b.over_min 
    from autumn a join maxminall b
    on a.MONTH || a.DAY = b.concat WHERE YEAR = ''' + str(y)



#Creates the max/min/all temp table
c = conn.cursor()
c.execute('''CREATE TEMPORARY TABLE maxminall AS Select MONTH || DAY concat, MONTH, DAY,MAX(TMAX) over_max, MIN(TMIN) over_min 
            from autumn GROUP BY 1, 2, 3 ORDER BY 2, 3''')
pd.read_sql('select * from maxminall', conn)

#Retrieves the range for the plots
rng = pd.read_sql('select MAX(TMAX) max, MIN(TMIN) min from autumn', conn)
mx, mn =(rng.loc[0, 'max'], rng.loc[0,'min'])

# New Version
def sql_dict():
    sql_results = {}
    for i in range(x - 1): 
        y = yr.loc[i,'YEAR']
        print(y)
        sql_results[str(y)] = {}
        sql_results[str(y)]['DATA'] = query('Select * from autumn where YEAR = '+ str(y))
        sql_results[str(y)]['DATA']['DATE'] = pd.to_datetime(sql_results[str(y)]['DATA']['DATE'])
        return sql_results

#makes the plot
def plot_maker(data, div):
    p = figure(x_axis_type='datetime',y_range=(mn,mx) ,width=500, plot_height=250, title=div)
    p.xgrid.grid_line_color=None
    p.ygrid.grid_line_color='red'
    p.ygrid.grid_line_alpha=0.5
    p.xaxis.axis_label = 'Time'
    p.yaxis.axis_label = "Temperature"
    p.line(data[div]['DATA'].DATE, data[div]['DATA'].TMAX, line_width=2, color='red')
    p.line(data[div]['DATA'].DATE, data[div]['DATA'].TMIN, line_width=2, color='blue')
    return p

# Grouping 
def data_grid(lst, col=2):
    output = []
    for i in range(0, len(lst), col):
        if (i + (col - 1)) >= len(lst):
            output.append([lst[i:]])
        else:
            output.append(lst[i:(i+(col))])
    return output


def display_all_plots(data):
    viz = []
    for i in data.keys():
        viz.append(plot_maker(data, i))
    show(gridplot(data_grid(viz)))

In [200]:
# # plot maker works -- displays the graph
# show(plot_maker(data=sql_results, div='2010'))

### -----
from bokeh.layouts import gridplot

viz = []
for i in sql_results.keys():
    viz.append(plot_maker(sql_results, i))
show(gridplot(data_grid(viz)))

In [39]:
import sqlite3

conn = sqlite3.connect('autumn1.db')

In [41]:
autumn.to_sql('autumn', conn)

In [42]:
query = "Select Distinct YEAR  from autumn"

yr = pd.read_sql(query, conn)

In [48]:
for i in yr.iteritems():
    print("And here is",i)

And here is ('YEAR', 0    2010
1    2011
2    2012
3    2013
4    2014
5    2015
6    2016
7    2017
8    2018
Name: YEAR, dtype: int64)


In [76]:
def query(sql):
    return pd.read_sql(sql, conn, index_col='index')

In [111]:
# misc function to turn strings into variables
x='buffalo'    
exec("%s = %d" % (x,2))
print(buffalo)

In [233]:
#To Test the Temp Table existence
pd.read_sql('select * from maxminall', conn)

,concat,MONTH,DAY,over_max,over_min
0,91,9,1,98,54
1,92,9,2,99,54
2,93,9,3,101,54
3,94,9,4,94,54
4,95,9,5,93,49
5,96,9,6,93,52
6,97,9,7,95,48
7,98,9,8,93,51
8,99,9,9,92,53
9,910,9,10,93,53


In [239]:
pwd

'/Users/ryanpierson'